Here, baseline values for the whole goodreads corpus are estimated and stored for usage in the evaluation of spoilernet on the whole corpus

In [1]:
import pandas as pd
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
#hide
from fastbook import *
from IPython.display import display,HTML
from fastai.text.all import *


df_raw = pd.read_json("data/raw/goodreads_reviews_spoiler.json.gz", compression='infer', lines=True)
df_raw.head()
df_raw1 = df_raw.iloc[range(0, len(df_raw), 2)]
#df_raw2 = df_raw.iloc[range(1, len(df_raw), 2)]
df_raw2 = df_raw.iloc[:200000] #spoilernet on full corpus

In [2]:
from tqdm import tqdm

b2b = {}
u2b = {}
for row in tqdm(df_raw1.values):
    bid = row[5]
    uid = row[0]
    for sent in row[2]:
        if bid in b2b.keys():
            b2b[bid].append(sent[0])
        else:
            b2b[bid] = [sent[0]]
        if uid in u2b.keys():
            u2b[uid].append(sent[0])
        else:
            u2b[uid] = [sent[0]]
for k in b2b.keys():
    val = np.sum(b2b[k]) / len(b2b[k])
    b2b[k] = val
for k in u2b.keys():
    val = np.sum(u2b[k]) / len(u2b[k])
    u2b[k] = val
len(u2b.keys()), len(b2b.keys())

100%|██████████| 689017/689017 [00:07<00:00, 97150.31it/s] 


(18690, 25467)

In [3]:
all_pos = []
all_ubs = []
all_bbs = []
all_sp = []
all_sents = []
nan_count = 0
gn_count = 0

for row in tqdm(df_raw2.values):
    for sent in row[2]:
        gn_count +=1
        if row[5] in b2b.keys():
            all_bbs.append(b2b[row[5]])
        else:
            all_bbs.append(float('nan'))
            nan_count += 1
        if row[0] in u2b.keys():
            all_ubs.append(u2b[row[0]])
        else:
            all_ubs.append(float('nan'))
            nan_count += 1
        all_sents.append(sent[1])
        all_sp.append(sent[0])
    for i in range(1, len(row[2])+1):
        all_pos.append(i/len(row[2]))
len(all_bbs) == len(all_ubs) == len(all_pos) == len(all_sp) == len(all_sents), gn_count -nan_count

100%|██████████| 200000/200000 [00:04<00:00, 49509.87it/s]


(True, 2456838)

In [4]:
import string

idfs = {}
dc = 0
for sent in tqdm(all_sents):
    if isinstance(sent, float):
        continue
    dc += 1
    sent = sent.translate(str.maketrans('', '', string.punctuation)).lower()
    s = set(sent.split())
    for el in s:
        if el in idfs.keys():
            idfs[el] += 1
        else:
            idfs[el] = 1

for k in idfs.keys():
    idfs[k] = math.log(float(dc)/ idfs[k])

100%|██████████| 2457349/2457349 [00:27<00:00, 88859.08it/s]


In [5]:
def compute_specificity(sents):
    all_words = {}
    all_sents = []
    w_count = 0
    for sent in sents:
        sent = sent[1].translate(str.maketrans('', '', string.punctuation)).lower()
        all_sents.append(sent)
        for w in sent.split():
            w_count += 1
            if w in all_words.keys():
                all_words[w] += 1
            else:
                all_words[w] = 1

    for w in all_words.keys():
        spec = (all_words[w]/w_count) * idfs[w]
        all_words[w] = spec
    return all_words, all_sents

all_idf = []
for row in tqdm(df_raw2.values):
    spec, sents = compute_specificity(row[2])
    for sent in sents:
        all_idf.append(np.average([spec[w] for w in sent.split()]))

  0%|          | 678/200000 [00:00<01:23, 2386.26it/s]/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 200000/200000 [00:55<00:00, 3582.54it/s]


In [6]:
len(all_bbs) == len(all_ubs) == len(all_pos) == len(all_sp) == len(all_idf)

True

In [7]:
df = pd.DataFrame({"sent":all_sents, "spoiler":all_sp, "book_bias":all_bbs, "user_bias":all_ubs, "position":all_pos, "specificity":all_idf})

In [8]:
df.to_csv('baseline_spoilernet.csv')

In [9]:
all_ubs

[0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.14314928425357873,
 0.1431492